### Wikipedia mapping from sql file

In [33]:
import re

def parse_sql_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sql_content = file.read()

    # Extract insert statements for wbc_entity_usage table
    insert_statements = re.findall(r"INSERT INTO `wbc_entity_usage` VALUES (.*?);", sql_content, re.DOTALL)

    mapping_data = {}
    for statement in insert_statements:
        # Remove the outer parentheses and split the values
        values = statement.strip()[1:-1].split("),(")
        
        for value in values:
            # Split the values by comma
            fields = value.split(",")
            wikidataId = fields[1].strip("'")
            prop_aspect = fields[2].strip("'") # only keep "statement"; 'D.en'

            # if wikidataId == "Q7930989":
            #     print(fields)
            
            if len(fields) == 4 and wikidataId.startswith('Q') and prop_aspect.startswith('D'):
                wikipediaId = fields[3]
                if wikidataId not in mapping_data:
                    mapping_data[wikidataId] = wikipediaId
    
    return mapping_data


In [34]:
# Example usage
file_path = 'enwiki-20240520-wbc_entity_usage.sql'
mapping_data = parse_sql_file(file_path)
print(len(mapping_data))

5821551


In [30]:
wiki_name = 'frwiki'
with open(wiki_name, 'w') as file:
    for wikidataId, wikipediaId in mapping_data.items():
        file.write(f"{wikipediaId},{wikidataId},{wiki_name}\n")

In [17]:
mapped_wiki_ents = set()
with open('enwiki', 'r') as enwiki:
    for line in enwiki:
        qid = line.strip().split(',')[1]
        mapped_wiki_ents.add(qid)